In [4]:
import tensorflow as tf
import numpy as np

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hyunj\Anaconda3\envs\tensorflow2\lib\site-packages\IPython\core\ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\hyunj\Anaconda3\envs\tensorflow2\lib\site-packages\IPython\core\ultratb.py", line 319, in wrapped
    return f(*args, **kwargs)
  File "C:\Users\hyunj\Anaconda3\envs\tensorflow2\lib\site-packages\IPython\core\ultratb.py", line 353, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "C:\Users\hyunj\Anaconda3\envs\tensorflow2\lib\inspect.py", line 1502, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "C:\Users\hyunj\Anaconda3\envs\tensorflow2\lib\inspect.py", line 1460, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "C:\Users\hyunj\Anaconda3\envs\tensorflow2\lib\inspect.py", line 696, in getsourcefile
    if getattr(ge

ModuleNotFoundError: No module named 'tensorflow.python.tools'; 'tensorflow.python' is not a package

#  하이퍼파라미터 설정-모델링할 때 사용자가 직접 세팅해주는 값


In [2]:
EPOCH=1000

# 네트워크 구조 정의

얕은 신경망 => 입력계층 2, 은닉계층(sigmoid activation) 128, 출력계층(softmax activation) 10

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1=tf.keras.layers.Dense(128, input_dim=2, activation="sigmoid")
        self.d2=tf.keras.layers.Dense(10, activation="softmax")

    def call(self,x,training=None, mask=None):
        x=self.d1(x)
        return self.d2(x)

In [4]:
@tf.function
def train_step(model,inputs,labels,loss_object, optimizer, train_loss,train_metric):
    with tf.GradientTape() as tape:  #Gradient 계산
        predictions=model(inputs)
        loss=loss_object(labels, predictions)
    gradients=tape.gradient(loss,model.trainable_variables)    #trainable variables => w, b
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # gradient 값을 통하여 W 값을 얻어내기 위한 그래프를 생성함
    train_loss(loss)
    train_metric(labels, predictions)

코드설명 

with as 로 파일을 열면 파일을 자동으로 close 해준다.
GradientTape()는 자동으로 미분을 해준다 (주어진 x에 대해서 자동으로 미분을 해준다)
tape.gradient(y, x) 를 하면 y를 x에 대해 미분해 준다.

tf.function을 함수에 붙여줄 경우, 여전히 다른 일반 함수들처럼 사용할 수 있습니다. 하지만 그래프 내에서 컴파일 되었을 때는 더 빠르게 실행하고, GPU나 TPU를 사용해서 작동하고, 세이브드모델(SavedModel)로 내보내는 것이 가능해집니다.



# 데이터셋 생성, 처리

In [ ]:
np.random.seed(0) # 코드를 여러번 실행해도 동일한 random 값들이 출력
pts=list()   # input 2 
labels=list()  #10

center_pts=np.random.uniform(-8.0,8.0,(10,2))
for idx, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt+np.random.randn(*center_pts.shape))
        labels.append(label)
        
        
pts=np.stack(pts,axis=0).astype(np.float32)
labels=np.stack(labels,axis=0)
        
train_ds=tf.data.Dataset.from_tensor_slices(pts,labels).shuffle(1000).batch(32)

In [ ]:
model=Mymodel()

In [ ]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
optimizer=tf.keras.optimizers.Adam()

In [ ]:
train_loss=tf.keras.metrics.Mean(name="train_loss")
train_accuracy=tf.keras.metrics.sparseCategoricalAccuracy(name="train_accuracy")


In [ ]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model,x,label,loss_object,optimizer,train_loss,train_accuracy)
    template="Epoch{}, loss{}, accuracy{}"
    print(template.format(epoch+1, train_loss.result, train_accuracy.result))